In [1]:
import pandas as pd
import numpy as np

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
import pywt

In [31]:
target = pd.read_csv('train/train_gts.csv')

In [33]:
target.head()

,record_name,myocard
0,00669_hr,0
1,00209_hr,0
2,00777_hr,0
3,01394_hr,0
4,01618_hr,0


In [43]:
def get_ecg(key):
  way = 'train/' + key + '.npy'
  ecgsig = np.load(way).transpose().reshape((60000,))
  x = np.linspace(0,  1,  num = 2048)
  data = np.sin(250 * np.pi * x**2)
  coefs = pywt.downcoef('a', data, 'db20', mode='symmetric', level=1)
  st='sym5'
  coeffs = pywt.wavedec(ecgsig, st, level=5)
  row = coeffs[0]
  return row

In [34]:
def into_one_table(keys):
  row = get_ecg(keys[0])
  row = np.insert(row, 0, int(keys[0].strip('_hr')))
  table = [row]
  for i in range(target.index.start + 1, target.index.stop):
    row = get_ecg(keys[i])
    row = np.insert(row, 0, int(keys[i].strip('_hr')))
    table = np.vstack([table, row])
  table = pd.DataFrame(table)
  return table

In [35]:
ECG_data = into_one_table(target.record_name)

In [37]:
ECG_data[0] = ECG_data[0].astype('int')
ECG_data = ECG_data.rename(columns={0: 'ecg_id'})

In [40]:
ECG_data = ECG_data.sort_values(by=['ecg_id'], ascending=True)

In [42]:
ECG_data.to_csv('new_ECG.csv')